<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_11/blob/Jason's-Code/Jasons_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def finite_diff(L, T, dx, dt, u, theta_init):
    '''
    The function takes input for space, time, intervals, velocity and
    initial concentration of pollution. Runs through the algorithm,
    modelling the change in concentration over time and space,
    returning graphs.
    '''

    # Calculates the number of discrete intervals
    Nx = int(round(L/dx, 2)) + 1
    Nt = int(round(T/dt, 2)) + 1

    # create the space grid
    x = np.linspace(0, L, Nx)

    # Construct arrays with placeholders for unknown concentrations
    new_conc = np.zeros(Nx)
    old_conc = np.zeros(Nx)

    # Construct arrays for matrix coefficients
    A = np.ones(Nx - 1)
    B = np.ones(Nx - 1)

    # Construct RHS array
    F = np.ones(Nx - 1)

    # Calculate the matrix coefficients
    a = 1/dt + u/dx
    A = a * A
    b = u/dx
    B = -b * B

    # start with initial concentration at t = 0
    old_conc = theta_init

    # loop over every time step
    for j in range(1, Nt + 1):

        # left boundary stays the same each step
        new_conc[0] = old_conc[0]

        # RHS at left boundary
        F[0] = (1/dt) * old_conc[0] + b * new_conc[0]

        # calculate the RHS at each x
        for i in range(1, Nx - 1):
            F[i] = (1/dt) * old_conc[i+1]

        # calculate the concentration using the finite difference formula
        for i in range(1, Nx - 1):
            new_conc[i] = (1/A[i-1]) * (F[i-1] - B[i-1] * new_conc[i-1])

        # right boundary condition
        new_conc[-1] = new_conc[-2]

        # generate plots for each timestep
        plt.plot(x, new_conc)
        plt.title(f"Pollutant Concentration at t = {j*dt} seconds")
        plt.xlabel("Distance along the river (m)")
        plt.ylabel("Pollutant concentration (µg/m³)")
        plt.grid(True)
        plt.show()

        # move to the next time step
        old_conc = new_conc.copy()



#TEST CASE 1

L = 20
dx = 0.2
T = 300
dt = 10
u = 0.1

theta_init_1 = np.zeros(int(round(L/dx, 2)) + 1)
theta_init_1[0] = 250   # pollutant at x = 0

finite_diff(L, T, dx, dt, u, theta_init_1)

#TEST CASE 2 — Using initial_conditions.csv


#reads CSV file
csv_data = pd.read_csv("initial_conditions.csv")

# extracst the columns
csv_distances = csv_data.iloc[:, 0]
csv_concentrations = csv_data.iloc[:, 1]

# interpolates CSV measurements to model grid
Nx_csv = int(round(L/dx, 2)) + 1
x_grid_csv = np.linspace(0, L, Nx_csv)

theta_init_2 = np.interp(x_grid_csv, csv_distances, csv_concentrations)

#runs the model with CSV data
finite_diff(L, T, dx, dt, u, theta_init_2)
